In [1]:
from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("symbol-vocab")
model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=2)
model.resize_token_embeddings(len(tokenizer))

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

Embedding(64614, 768)

In [2]:
from glob import glob
from datasets import Dataset

import re
import pandas as pd

from parse import normalize_except_compony

def data_loading(url):
    with open(url, 'r', encoding='utf-8') as f:
        df = pd.read_json(f)
        data = df.copy()
        data = data.loc[df['sentiment'].notnull()]
        data['body'] = data['body'].apply(normalize_except_compony, args=(tokenizer, ))
        data['sentiment'] = pd.Categorical(data['sentiment'])
        data['label'] = data['sentiment'].cat.codes
        data = data.rename(columns={'sentiment': 'labels', 'body': 'sentense'})

        # symbols = set()
        # for symbol_list in data['sentense'].str.findall(r'\$[A-Z]+'):
        #     for symbol in symbol_list:
        #         symbols.add(symbol)
        return data

data_url = '../crawler/stock/data/**.json'
url = glob(data_url)[-1]
data = data_loading(url)

dataset = Dataset.from_pandas(data.loc[:, ['label', 'sentense']])
dataset = dataset.remove_columns('__index_level_0__')

# special_tokens_dict = {'additional_special_tokens': list(symbols)}
# tokenizer.add_special_tokens(special_tokens_dict)
# model.resize_token_embeddings(len(tokenizer))

In [3]:
def encode(example):
    sentense = example['sentense']

    result = tokenizer(
        sentense,
        padding='max_length',
        truncation=True,
    )
    return result

encoded_dataset = dataset.map(encode, batched=True)
encoded_dataset = encoded_dataset.train_test_split(test_size=0.2)
# print(encoded_dataset[0]['sentense'])
# print(encoded_dataset[0]['input_ids'])
# type(encoded_dataset[0]['labels'])

100%|██████████| 16/16 [00:04<00:00,  3.41ba/s]


In [4]:
from transformers import TrainingArguments, Trainer
from datasets import load_metric

import numpy as np
import wandb

run_name = "roberta-classifier-5-eopch"
%env WANDB_PROJECT=Stocktwit_sentiment_analysis
training_args = TrainingArguments(
    output_dir=f"{run_name}-trainer",
    evaluation_strategy="steps",
    eval_steps=100,
    num_train_epochs=5,
    save_steps=5000,
    per_device_train_batch_size=4,
    report_to="wandb",
    run_name=run_name
)

recall_metric = load_metric("recall")
precision_metric = load_metric("precision")
accuracy_metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    recall_result = recall_metric.compute(predictions=predictions, references=labels)
    precision_result = precision_metric.compute(predictions=predictions, references=labels)
    accuracy_result = accuracy_metric.compute(predictions=predictions, references=labels)
    return {**recall_result, **precision_result, **accuracy_result}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['test'],
    compute_metrics=compute_metrics,
)

trainer.train()

wandb.finish()

env: WANDB_PROJECT=Stocktwit_sentiment_analysis


The following columns in the training set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentense. If sentense are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
C:\Users\douli\anaconda3\envs\stock-nlp\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 12409
  Num Epochs = 5
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 15515
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Failed to detect the name of this notebook, you can se

  1%|          | 100/15515 [00:12<26:07,  9.83it/s]The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentense. If sentense are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3103
  Batch size = 8

  1%|          | 102/15515 [00:29<16:08:12,  3.77s/it]

{'eval_loss': 0.537808358669281, 'eval_recall': 1.0, 'eval_precision': 0.7831131163390267, 'eval_accuracy': 0.7831131163390267, 'eval_runtime': 17.4675, 'eval_samples_per_second': 177.644, 'eval_steps_per_second': 22.213, 'epoch': 0.03}


  1%|▏         | 200/15515 [00:39<26:32,  9.62it/s]   The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentense. If sentense are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3103
  Batch size = 8

  1%|▏         | 202/15515 [00:57<16:11:30,  3.81s/it]

...

                                                       
100%|█████████▉| 15501/15515 [1:08:55<00:36,  2.62s/it]

{'eval_loss': 0.5234593749046326, 'eval_recall': 1.0, 'eval_precision': 0.7831131163390267, 'eval_accuracy': 0.7831131163390267, 'eval_runtime': 16.533, 'eval_samples_per_second': 187.685, 'eval_steps_per_second': 23.468, 'epoch': 5.0}


100%|██████████| 15515/15515 [1:08:57<00:00,  3.34it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 15515/15515 [1:08:57<00:00,  3.75it/s]


{'train_runtime': 4139.8506, 'train_samples_per_second': 14.987, 'train_steps_per_second': 3.748, 'train_loss': 0.5491651629080052, 'epoch': 5.0}



eval/accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/loss,▄▁█▁▁▂▆▂▁▃▄▃▁▁▁▁▂▁▁▁▁▁▁▃▂▁▁▁▃▂▁▁▂▁▂▁▁▁▁▁
eval/precision,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/recall,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/runtime,██▂▂▃▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▁▁
eval/samples_per_second,▁▁▇▇▆▆▇▇▆▆▇█▇████████████████▇██████▇▇██
eval/steps_per_second,▁▁▇▇▆▆▆▇▆▆▇█▇████████████████▇██████▇▇██
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,███▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▄▃▃▃▃▂▂▂▂▁▁▁
train/loss,█▄▂▅▅▄▃▃▃▃▅▄▃▃▃▃▃▂▂▄▃▁▃▃▂▃▂▃▃▃▃
